# 📜 OCR/HTR with Gemini AI

Extract text from **PDFs** and **images** using Google's Gemini AI. Supports both printed documents (OCR) and handwritten manuscripts (HTR) in multiple languages.

In [ ]:
# Install required packages
!pip install -q google-genai PyPDF2 Pillow ipywidgets ipyfilechooser

# Import necessary libraries
import os
import io
import shutil
from pathlib import Path
from google.colab import files
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from google import genai
from google.genai import types
from PyPDF2 import PdfReader, PdfWriter
from PIL import Image

# ============================================
# SUPPORTED FILE FORMATS
# ============================================

# Image formats supported by Gemini
SUPPORTED_IMAGE_FORMATS = {'.jpg', '.jpeg', '.png', '.webp', '.heic', '.heif'}
SUPPORTED_PDF_FORMAT = {'.pdf'}
ALL_SUPPORTED_FORMATS = SUPPORTED_IMAGE_FORMATS | SUPPORTED_PDF_FORMAT

# MIME type mapping
MIME_TYPES = {
    '.jpg': 'image/jpeg',
    '.jpeg': 'image/jpeg',
    '.png': 'image/png',
    '.webp': 'image/webp',
    '.heic': 'image/heic',
    '.heif': 'image/heif',
    '.pdf': 'application/pdf'
}

# ============================================
# CREATE FOLDER STRUCTURE
# ============================================

FOLDERS = {
    'input': 'input_files',  # For PDFs and images
    'results': 'results',
    'prompts': 'prompts',
    'log': 'logs'
}

for folder_name, folder_path in FOLDERS.items():
    os.makedirs(folder_path, exist_ok=True)

# ============================================
# GOOGLE DRIVE CONFIGURATION
# ============================================

DRIVE_MOUNTED = False
DRIVE_BASE_PATH = '/content/drive/My Drive'
DRIVE_OUTPUT_FOLDER = 'Colab_OCR_Results'

def mount_google_drive():
    """Mount Google Drive if not already mounted."""
    global DRIVE_MOUNTED
    if not DRIVE_MOUNTED:
        try:
            drive.mount('/content/drive')
            DRIVE_MOUNTED = True
            print("✅ Google Drive mounted successfully!")
            return True
        except Exception as e:
            print(f"❌ Failed to mount Google Drive: {e}")
            return False
    return True

def get_drive_output_folder():
    """Get or create the output folder in Google Drive."""
    if not DRIVE_MOUNTED:
        return None
    output_path = os.path.join(DRIVE_BASE_PATH, DRIVE_OUTPUT_FOLDER)
    os.makedirs(output_path, exist_ok=True)
    return output_path


# ============================================
# CREATE PROMPT FILES
# ============================================

PROMPT_CONTENT = {
    "ocr_universal.md": '''# Universal OCR/HTR System Prompt

You are a high-precision OCR/HTR system engineered to produce research-grade, archival-quality text extraction from any document type in any language. Your output directly supports academic research and archival preservation.

## Core Principles

1. **Research-Grade Accuracy:** TRANSCRIBE every single word and character with absolute precision. Work character by character, word by word, line by line to minimize Character Error Rate (CER) and Word Error Rate (WER).
2. **Historical Authenticity:** PRESERVE the text exactly as written. RETAIN all spelling variations, grammatical structures, and punctuation. DO NOT normalize, modernize, or correct the text.
3. **Systematic Zone Analysis:** IDENTIFY and PROCESS distinct content zones in their precise reading order.
4. **Pure Archival Transcription:** DELIVER exact transcription only – no summarization, interpretation, or omissions.
5. **Multi-Script Support:** HANDLE all writing systems with equal precision.
6. **Mixed Content Processing:** TRANSCRIBE both printed and handwritten text, clearly indicating handwritten sections.

## Reading Direction
- Left-to-right, top-to-bottom for Western documents
- Right-to-left for Arabic, Hebrew, Persian, Urdu
- Appropriate direction for other writing systems

## Text Processing
- EXECUTE de-hyphenation: remove end-of-line hyphens (e.g., `ana-\\nlyse` → `analyse`)
- PRESERVE legitimate compound hyphens (e.g., `arc-en-ciel`)
- REPLICATE all diacritical marks exactly (é, ñ, ü, ç, etc.)
- MERGE semantically linked lines within the same paragraph
- ENFORCE double newline between paragraphs

## Handwritten Text
- MARK handwritten sections: `[HANDWRITTEN: text]`
- USE `[UNCERTAIN: text]` for unclear words
- INDICATE `[ILLEGIBLE]` for unreadable text

## Output Requirements
- DELIVER pure transcribed text only
- EXCLUDE all commentary or explanations
- PRESERVE original language(s) without translation
''',

    "htr_french.md": '''# HTR System Prompt for French Handwritten Documents

You are a high-precision HTR (Handwritten Text Recognition) system specialized in French-language handwritten documents, engineered for research-grade, archival-quality text extraction.

## Core Principles
1. **Research-Grade Accuracy:** TRANSCRIBE every word and character with absolute precision.
2. **Historical Authenticity:** PRESERVE text exactly as written – DO NOT normalize or correct.
3. **Pure Archival Transcription:** DELIVER exact transcription only.

## French-Specific Rules
- IMPLEMENT French spacing: ` : `, ` ; `, ` ! `, ` ? ` (space before punctuation)
- EXECUTE de-hyphenation: `cho-\\nrégraphiques` → `chorégraphiques`
- PRESERVE legitimate compounds: `arc-en-ciel`
- REPLICATE all diacritics: é, è, ê, ë, à, â, ù, û, ô, î, ï, ç, œ, æ

## Text Processing
- MERGE lines within the same paragraph
- ENFORCE double newline (`\\n\\n`) between paragraphs
- MARK uncertain readings with [?]

## Output Requirements
- DELIVER pure transcribed French text only
- EXCLUDE all commentary
- PRESERVE original spelling and grammar exactly
''',

    "htr_arabic.md": '''# HTR System Prompt for Arabic Handwritten Manuscripts

You are a high-precision HTR system specialized in Arabic-language handwritten manuscripts, engineered for research-grade, archival-quality text extraction.

## Core Principles
1. **Research-Grade Accuracy:** TRANSCRIBE every word and character with absolute precision.
2. **Historical Authenticity:** PRESERVE text exactly as written – DO NOT normalize or correct.
3. **Pure Archival Transcription:** DELIVER exact transcription only.

## Arabic-Specific Rules
- READ right-to-left, following traditional manuscript conventions
- REPLICATE all diacritical marks (tashkeel, harakat) when present
- PRESERVE ligatures and connected letter forms
- RESPECT traditional manuscript orthography

## Text Processing
- MERGE lines within the same paragraph
- ENFORCE double newline (`\\n\\n`) between paragraphs
- MARK uncertain readings with [?]

## Output Requirements
- DELIVER pure transcribed Arabic text only
- EXCLUDE all commentary
- PRESERVE original spelling and historical orthography
''',

    "htr_multilingual.md": '''# HTR System Prompt for Multilingual Handwritten Documents

You are a high-precision HTR system specialized in multilingual handwritten documents, engineered for research-grade, archival-quality text extraction.

## Language Detection Protocol
Before transcription, IDENTIFY:
1. **Writing system(s):** Latin, Arabic, Cyrillic, Greek, Hebrew, Chinese, Japanese, Korean, Devanagari, etc.
2. **Language(s):** Based on vocabulary, grammar, diacritics
3. **Text direction:** LTR, RTL, or vertical

## Output Header
BEGIN transcription with:
```
[LANGUAGE DETECTED: <language>]
[WRITING SYSTEM: <script>]
[TEXT DIRECTION: <direction>]
```

## Language-Specific Processing

### Latin Scripts (French, English, Spanish, German, etc.)
- Execute de-hyphenation for end-of-line breaks
- Apply language-specific spacing rules
- Preserve all diacritics (é, ñ, ö, ą, etc.)

### Arabic Script
- Read right-to-left
- Preserve tashkeel when present
- Maintain ligatures and connected forms

### Cyrillic Script
- Preserve hard/soft signs (ъ, ь)
- Maintain special characters (ё, є, і, ї)

### Other Scripts
- Use correct Unicode characters
- Preserve all diacritics and vowel marks

## Output Requirements
- BEGIN with language detection header
- DELIVER pure transcribed text
- NOTE language switches with [LANGUAGE SWITCH: <new language>]
'''
}

# Write prompt files to disk
for filename, content in PROMPT_CONTENT.items():
    filepath = os.path.join(FOLDERS['prompts'], filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)

print("✅ Setup complete!")
print()
print("📁 Folder structure created:")
print(f"   ├── 📂 {FOLDERS['input']}/      ← Upload PDFs or images here")
print(f"   ├── 📂 {FOLDERS['results']}/         ← Output text files")
print(f"   └── 📂 {FOLDERS['prompts']}/         ← System prompts")
print()
print("📄 Available prompts:")
for prompt_name in PROMPT_CONTENT.keys():
    print(f"   • {prompt_name}")
print()
print("🖼️ Supported formats:")
print(f"   • Images: {', '.join(sorted(SUPPORTED_IMAGE_FORMATS))}")
print(f"   • Documents: .pdf")

## Step 2: Enter Your API Key 🔑

Enter your Google Gemini API key below.

**Recommended:** Add your API key to **Colab Secrets** (🔑 icon in left sidebar) with the name `GEMINI_API_KEY` for automatic loading.

**Don't have one?** Get it free at: https://aistudio.google.com/apikey

In [ ]:
# ============================================
# API KEY CONFIGURATION
# ============================================
# Best practice: Use Colab Secrets (recommended)
# 1. Click the 🔑 key icon in the left sidebar
# 2. Add a secret named "GEMINI_API_KEY"
# 3. Paste your API key and enable notebook access

api_key_input = widgets.Password(
    placeholder='Paste your API key here (or use Colab Secrets)',
    description='API Key:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '80px'}
)

api_key_status = widgets.HTML(value="")

def validate_api_key(change):
    if len(change['new']) > 20:
        api_key_status.value = "<span style='color: green;'>✅ API key entered</span>"
    else:
        api_key_status.value = "<span style='color: orange;'>⏳ Please enter your full API key</span>"

api_key_input.observe(validate_api_key, names='value')

# Try to load from Colab Secrets (best practice)
try:
    from google.colab import userdata
    secret_key = userdata.get('GEMINI_API_KEY')
    if secret_key:
        api_key_input.value = secret_key
        api_key_status.value = "<span style='color: green;'>✅ API key loaded from Colab Secrets (recommended)</span>"
except Exception:
    api_key_status.value = "<span style='color: blue;'>💡 Tip: Add GEMINI_API_KEY to Colab Secrets for secure auto-loading</span>"

display(HTML("<b>Enter your Gemini API key:</b>"))
display(api_key_input)
display(api_key_status)
display(HTML("<br>"))
display(HTML("""
<details>
<summary>🔐 <b>How to use Colab Secrets (recommended)</b></summary>
<ol>
<li>Click the 🔑 <b>Secrets</b> icon in the left sidebar</li>
<li>Click <b>+ Add new secret</b></li>
<li>Name: <code>GEMINI_API_KEY</code></li>
<li>Value: Your API key (starts with <code>AIza...</code>)</li>
<li>Toggle <b>Notebook access</b> ON</li>
<li>Re-run this cell to auto-load</li>
</ol>
</details>
"""))

## Step 2.5: Connect Google Drive (Optional) ☁️

Mount your Google Drive to **load files directly** and **auto-save OCR results**.

**Benefits:**
- 📂 Load PDFs/images directly from your Drive
- 💾 OCR results auto-saved to Drive (survives browser disconnection!)
- 🔄 Access your files across sessions

In [ ]:
# ============================================
# GOOGLE DRIVE CONNECTION
# ============================================

drive_status = widgets.HTML(value="")
drive_save_enabled = widgets.Checkbox(
    value=True,
    description='Auto-save OCR results to Google Drive',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px'),
    disabled=True
)

drive_folder_input = widgets.Text(
    value='Colab_OCR_Results',
    description='Save folder:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px'),
    disabled=True
)

def connect_drive(b):
    global DRIVE_MOUNTED, DRIVE_OUTPUT_FOLDER
    drive_status.value = "<span style='color: blue;'>🔄 Connecting to Google Drive...</span>"
    
    if mount_google_drive():
        DRIVE_MOUNTED = True
        drive_save_enabled.disabled = False
        drive_folder_input.disabled = False
        
        drive_status.value = f"""
        <div style='background: #e8f5e9; padding: 10px; border-radius: 5px; margin-top: 10px;'>
        ✅ <b>Google Drive connected!</b><br>
        📂 Your Drive is mounted at: <code>/content/drive/My Drive/</code><br>
        💾 OCR results will be saved to: <code>{DRIVE_OUTPUT_FOLDER}/</code>
        </div>
        """
    else:
        drive_status.value = "<span style='color: red;'>❌ Failed to connect. You can still use local upload/download.</span>"

def update_drive_folder(change):
    global DRIVE_OUTPUT_FOLDER
    DRIVE_OUTPUT_FOLDER = change['new'].strip() or 'Colab_OCR_Results'

drive_folder_input.observe(update_drive_folder, names='value')

connect_button = widgets.Button(
    description='☁️ Connect Google Drive',
    button_style='primary',
    layout=widgets.Layout(width='220px', height='40px')
)
connect_button.on_click(connect_drive)

display(connect_button)
display(drive_status)
display(HTML("<br>"))
display(HTML("<b>Auto-save settings (enabled after connecting):</b>"))
display(drive_save_enabled)
display(drive_folder_input)
display(HTML("<i>💡 OCR results will be saved to: My Drive/[folder name]/</i>"))

## Step 3: Select Your Files 📁

Choose how to load your PDF/image files:
- **📤 Local Upload**: Upload files from your computer
- **☁️ Google Drive**: Select files from your connected Drive

**Supported formats:** PDF, JPG, PNG, WEBP, HEIC

In [ ]:
# ============================================
# FILE SOURCE SELECTION (Local or Drive)
# ============================================

uploaded_files = []
upload_status = widgets.HTML(value="")
drive_browser_status = widgets.HTML(value="")

# ---- LOCAL UPLOAD TAB ----
def upload_files(b):
    global uploaded_files
    upload_status.value = "<span style='color: blue;'>📤 Upload dialog opened... Select your file(s)</span>"
    
    try:
        uploaded = files.upload()
        
        if uploaded:
            uploaded_files = []
            valid_files = []
            invalid_files = []
            
            for filename, content in uploaded.items():
                ext = Path(filename).suffix.lower()
                if ext in ALL_SUPPORTED_FORMATS:
                    filepath = os.path.join(FOLDERS['input'], filename)
                    with open(filepath, 'wb') as f:
                        f.write(content)
                    uploaded_files.append(filepath)
                    valid_files.append((filename, ext))
                else:
                    invalid_files.append(filename)
            
            status_html = ""
            if valid_files:
                status_html += f"<span style='color: green;'>✅ Uploaded {len(valid_files)} file(s):</span><br>"
                for f, ext in valid_files:
                    icon = "🖼️" if ext in SUPPORTED_IMAGE_FORMATS else "📄"
                    status_html += f"&nbsp;&nbsp;&nbsp;{icon} {f}<br>"
            if invalid_files:
                status_html += f"<span style='color: red;'>❌ Skipped {len(invalid_files)} unsupported:</span><br>"
                for f in invalid_files:
                    status_html += f"&nbsp;&nbsp;&nbsp;⚠️ {f}<br>"
            
            upload_status.value = status_html
        else:
            upload_status.value = "<span style='color: orange;'>⚠️ No files uploaded</span>"
    except Exception as e:
        upload_status.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"

upload_button = widgets.Button(
    description='📁 Upload PDF or Image Files',
    button_style='primary',
    layout=widgets.Layout(width='280px', height='40px')
)
upload_button.on_click(upload_files)

local_upload_tab = widgets.VBox([
    upload_button,
    upload_status,
    widgets.HTML("<br><i>💡 Supported: PDF, JPG, PNG, WEBP, HEIC</i>")
])

# ---- GOOGLE DRIVE TAB ----
try:
    from ipyfilechooser import FileChooser
    FILECHOOSER_AVAILABLE = True
except ImportError:
    FILECHOOSER_AVAILABLE = False

drive_selected_files = []
selected_files_display = widgets.HTML(value="<i>No files selected yet</i>")

def update_selected_display():
    if not drive_selected_files:
        selected_files_display.value = "<i>No files selected yet</i>"
        return
    
    html = f"<b>Selected {len(drive_selected_files)} file(s):</b><br>"
    for filepath in drive_selected_files:
        filename = Path(filepath).name
        ext = Path(filepath).suffix.lower()
        icon = "🖼️" if ext in SUPPORTED_IMAGE_FORMATS else "📄"
        html += f"&nbsp;&nbsp;{icon} {filename}<br>"
    selected_files_display.value = html

def on_file_selected(chooser):
    if chooser.selected:
        filepath = chooser.selected
        ext = Path(filepath).suffix.lower()
        if ext in ALL_SUPPORTED_FORMATS:
            if filepath not in drive_selected_files:
                drive_selected_files.append(filepath)
                update_selected_display()
                drive_browser_status.value = f"<span style='color: green;'>✅ Added: {Path(filepath).name}</span>"
            else:
                drive_browser_status.value = "<span style='color: orange;'>⚠️ File already selected</span>"
        else:
            drive_browser_status.value = f"<span style='color: red;'>❌ Unsupported format: {ext}</span>"

def clear_selection(b):
    global drive_selected_files
    drive_selected_files = []
    update_selected_display()
    drive_browser_status.value = "<span style='color: blue;'>🗑️ Selection cleared</span>"

def confirm_drive_selection(b):
    global uploaded_files
    
    if not drive_selected_files:
        drive_browser_status.value = "<span style='color: red;'>❌ Please select at least one file</span>"
        return
    
    uploaded_files = []
    errors = []
    
    for src_path in drive_selected_files:
        try:
            filename = Path(src_path).name
            dest_path = os.path.join(FOLDERS['input'], filename)
            shutil.copy2(src_path, dest_path)
            uploaded_files.append(dest_path)
        except Exception as e:
            errors.append(f"{Path(src_path).name}: {e}")
    
    if uploaded_files:
        status_html = f"<span style='color: green;'>✅ Ready to process {len(uploaded_files)} file(s)!</span>"
        if errors:
            status_html += f"<br><span style='color: red;'>⚠️ Errors: {len(errors)}</span>"
        drive_browser_status.value = status_html
    else:
        drive_browser_status.value = "<span style='color: red;'>❌ Failed to prepare files</span>"

if FILECHOOSER_AVAILABLE and DRIVE_MOUNTED:
    file_chooser = FileChooser(
        path='/content/drive/My Drive',
        filename='',
        title='<b>📂 Browse Google Drive:</b>',
        show_hidden=False,
        select_default=False,
        show_only_dirs=False,
        filter_pattern=['*.pdf', '*.jpg', '*.jpeg', '*.png', '*.webp', '*.heic', '*.heif']
    )
    file_chooser.register_callback(on_file_selected)
    
    clear_btn = widgets.Button(description='🗑️ Clear Selection', button_style='warning', layout=widgets.Layout(width='150px'))
    clear_btn.on_click(clear_selection)
    
    confirm_btn = widgets.Button(description='✅ Confirm & Proceed', button_style='success', layout=widgets.Layout(width='180px'))
    confirm_btn.on_click(confirm_drive_selection)
    
    drive_tab = widgets.VBox([
        widgets.HTML("<p><i>Navigate folders and click files to add them to selection.</i></p>"),
        file_chooser,
        widgets.HTML("<hr style='margin: 10px 0;'>"),
        selected_files_display,
        widgets.HBox([confirm_btn, clear_btn]),
        drive_browser_status
    ])

elif DRIVE_MOUNTED:
    # Fallback simple browser
    drive_folder_path = widgets.Text(
        value='/content/drive/My Drive/',
        description='Folder:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='450px')
    )
    
    drive_file_list = widgets.SelectMultiple(
        options=[],
        description='Files:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='500px', height='180px')
    )
    
    def refresh_drive_files(b):
        folder = drive_folder_path.value
        if not os.path.exists(folder):
            drive_browser_status.value = "<span style='color: red;'>❌ Folder not found</span>"
            return
        
        supported_files = []
        try:
            for f in os.listdir(folder):
                if os.path.isfile(os.path.join(folder, f)):
                    ext = Path(f).suffix.lower()
                    if ext in ALL_SUPPORTED_FORMATS:
                        icon = "🖼️" if ext in SUPPORTED_IMAGE_FORMATS else "📄"
                        supported_files.append(f"{icon} {f}")
        except Exception as e:
            drive_browser_status.value = f"<span style='color: red;'>❌ Error: {e}</span>"
            return
        
        drive_file_list.options = sorted(supported_files)
        if supported_files:
            drive_browser_status.value = f"<span style='color: green;'>✅ Found {len(supported_files)} file(s)</span>"
        else:
            drive_browser_status.value = "<span style='color: orange;'>⚠️ No supported files found</span>"
    
    def select_drive_files(b):
        global uploaded_files
        
        selected = drive_file_list.value
        if not selected:
            drive_browser_status.value = "<span style='color: red;'>❌ Please select files first</span>"
            return
        
        uploaded_files = []
        folder = drive_folder_path.value
        
        for display_name in selected:
            filename = display_name.split(' ', 1)[1] if ' ' in display_name else display_name
            try:
                src_path = os.path.join(folder, filename)
                dest_path = os.path.join(FOLDERS['input'], filename)
                shutil.copy2(src_path, dest_path)
                uploaded_files.append(dest_path)
            except Exception:
                pass
        
        if uploaded_files:
            drive_browser_status.value = f"<span style='color: green;'>✅ Ready: {len(uploaded_files)} file(s)</span>"
        else:
            drive_browser_status.value = "<span style='color: red;'>❌ Failed to prepare files</span>"
    
    refresh_btn = widgets.Button(description='🔄 Refresh Files', button_style='info', layout=widgets.Layout(width='130px'))
    refresh_btn.on_click(refresh_drive_files)
    
    select_btn = widgets.Button(description='✅ Select Files', button_style='success', layout=widgets.Layout(width='130px'))
    select_btn.on_click(select_drive_files)
    
    drive_tab = widgets.VBox([
        widgets.HTML("<p><b>📂 Enter folder path and browse files:</b></p>"),
        widgets.HBox([drive_folder_path, refresh_btn]),
        drive_file_list,
        select_btn,
        drive_browser_status
    ])

else:
    drive_tab = widgets.VBox([
        widgets.HTML("""
        <div style='padding: 20px; background: #fff3e0; border-radius: 8px; margin: 10px 0;'>
        <h4>☁️ Google Drive Not Connected</h4>
        <p>To browse files from Google Drive:</p>
        <ol>
        <li>Go back to <b>Step 2.5</b></li>
        <li>Click <b>"☁️ Connect Google Drive"</b></li>
        <li>Authorize access when prompted</li>
        <li>Return here and <b>re-run this cell</b></li>
        </ol>
        </div>
        """)
    ])

file_source_tabs = widgets.Tab()
file_source_tabs.children = [local_upload_tab, drive_tab]
file_source_tabs.set_title(0, '📤 Local Upload')
file_source_tabs.set_title(1, '☁️ Google Drive')

display(file_source_tabs)


## Step 4: OCR/HTR Settings 🎛️

Select the AI model and document type. Use **HTR prompts** for handwritten manuscripts.

In [ ]:
# ============================================
# SETTINGS WIDGETS
# ============================================

# Model selection (Gemini 3 Pro only supports thinking_level, Flash supports budget)
model_dropdown = widgets.Dropdown(
    options=[
        ('Gemini 3 Pro (Highest quality)', 'gemini-3-pro-preview'),
        ('Gemini 3 Flash (Faster, good quality)', 'gemini-3-flash-preview'),
    ],
    value='gemini-3-pro-preview',
    description='AI Model:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='450px')
)

# Document type / prompt selection
prompt_dropdown = widgets.Dropdown(
    options=[
        ('🔤 Universal OCR (printed documents, auto-detect language)', 'ocr_universal.md'),
        ('✍️ HTR French (handwritten French manuscripts)', 'htr_french.md'),
        ('✍️ HTR Arabic (handwritten Arabic manuscripts)', 'htr_arabic.md'),
        ('✍️ HTR Multilingual (handwritten, auto-detect language)', 'htr_multilingual.md'),
    ],
    value='ocr_universal.md',
    description='Document Type:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

# Custom prompt option
use_custom_prompt = widgets.Checkbox(
    value=False,
    description='Use custom prompt instead',
    style={'description_width': 'initial'}
)

custom_prompt_text = widgets.Textarea(
    placeholder='Enter your custom OCR/HTR instructions here...',
    layout=widgets.Layout(width='500px', height='150px'),
    disabled=True
)

def toggle_custom_prompt(change):
    custom_prompt_text.disabled = not change['new']
    prompt_dropdown.disabled = change['new']

use_custom_prompt.observe(toggle_custom_prompt, names='value')

# Model info display
model_info = widgets.HTML(value="")

def update_model_info(change):
    model = change['new']
    if "3-pro" in model:
        model_info.value = """
        <div style='background: #e8f5e9; padding: 10px; border-radius: 5px; margin: 5px 0;'>
        🧠 <b>Gemini 3 Pro</b>: Thinking level LOW, Temperature 0.1<br>
        🖼️ Images: High resolution (1120 tokens/image)<br>
        📄 PDFs: High resolution for best OCR
        </div>
        """
    else:
        model_info.value = """
        <div style='background: #e3f2fd; padding: 10px; border-radius: 5px; margin: 5px 0;'>
        ⚡ <b>Gemini 3 Flash</b>: Thinking level MINIMAL, Temperature 0.2<br>
        🖼️ Images: High resolution with Pan & Scan<br>
        📄 PDFs: Optimized for speed
        </div>
        """

model_dropdown.observe(update_model_info, names='value')
update_model_info({'new': model_dropdown.value})

display(HTML("<h3>🤖 Select AI Model</h3>"))
display(model_dropdown)
display(model_info)

display(HTML("<h3>📜 Select Document Type</h3>"))
display(prompt_dropdown)

display(HTML("<br>"))
display(use_custom_prompt)
display(custom_prompt_text)

## Step 5: Start OCR Processing 🚀

Click the button below to start processing your PDF file(s).

In [ ]:
# ============================================
# OCR/HTR ENGINE
# ============================================

class GeminiOCR:
    """
    A high-precision OCR/HTR system using Google's Gemini model.
    Supports PDFs and image files with optimized settings per model.
    """

    def __init__(self, api_key: str, model_name: str, prompt_file: str = None, custom_prompt: str = None):
        self.client = genai.Client(api_key=api_key)
        self.model_name = model_name
        self.prompt_file = prompt_file
        self.custom_prompt = custom_prompt
        self.generation_config = self._setup_generation_config()
        
    def _get_system_instruction(self):
        """Load system instruction from prompt file or custom prompt."""
        if self.custom_prompt:
            return self.custom_prompt
        
        if self.prompt_file:
            prompt_path = os.path.join(FOLDERS['prompts'], self.prompt_file)
            try:
                with open(prompt_path, 'r', encoding='utf-8') as f:
                    return f.read()
            except Exception as e:
                print(f"❌ Error reading prompt file: {e}")
                raise
        
        # Default fallback
        return "Perform OCR on this document. Extract all text accurately."
        
    def _setup_generation_config(self):
        """Configure generation settings based on model type.
        
        Gemini 3 Pro: Uses thinking_level ('low'), temperature 0.1
        Gemini 3 Flash: Uses thinking_level ('MINIMAL'), temperature 0.2
        
        Both use MEDIA_RESOLUTION_HIGH for best OCR quality.
        """
        config_params = {
            "top_p": 0.95,
            "top_k": 40,
            "max_output_tokens": 65535,
            "response_mime_type": "text/plain",
            "system_instruction": self._get_system_instruction(),
            "media_resolution": types.MediaResolution.MEDIA_RESOLUTION_HIGH,
        }
        
        if "3-pro" in self.model_name.lower():
            # Gemini 3 Pro: Use thinking_level 'LOW', low temperature
            config_params["temperature"] = 0.1
            config_params["thinking_config"] = types.ThinkingConfig(thinking_level="LOW")
        else:
            # Gemini 3 Flash: Use thinking_level 'MINIMAL', slightly higher temperature
            config_params["temperature"] = 0.2
            config_params["thinking_config"] = types.ThinkingConfig(thinking_level="MINIMAL")
        
        # Safety settings for archival content
        config_params["safety_settings"] = [
            types.SafetySetting(
                category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
                threshold=types.HarmBlockThreshold.BLOCK_NONE
            ),
            types.SafetySetting(
                category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                threshold=types.HarmBlockThreshold.BLOCK_NONE
            ),
            types.SafetySetting(
                category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                threshold=types.HarmBlockThreshold.BLOCK_NONE
            ),
            types.SafetySetting(
                category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=types.HarmBlockThreshold.BLOCK_NONE
            )
        ]
        
        return types.GenerateContentConfig(**config_params)

    def extract_pdf_page(self, pdf_path, page_number):
        """Extract a single page from PDF as bytes."""
        try:
            reader = PdfReader(str(pdf_path))
            writer = PdfWriter()
            writer.add_page(reader.pages[page_number])
            output_buffer = io.BytesIO()
            writer.write(output_buffer)
            output_buffer.seek(0)
            return output_buffer.getvalue()
        except Exception as e:
            print(f"❌ Error extracting page {page_number + 1}: {e}")
            raise

    def get_pdf_page_count(self, pdf_path):
        """Get number of pages in PDF."""
        try:
            reader = PdfReader(str(pdf_path))
            return len(reader.pages)
        except Exception as e:
            print(f"❌ Error reading PDF: {e}")
            raise

    def process_content(self, content_bytes, mime_type, label=""):
        """Process content (PDF page or image) using Gemini API."""
        try:
            print(f"   └─ 📄 Processing {label}...")
            
            content_part = types.Part.from_bytes(
                data=content_bytes,
                mime_type=mime_type
            )
            
            # Simple user prompt - system instruction is in config
            user_prompt = "Please perform complete OCR transcription."
            
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=[content_part, user_prompt],
                config=self.generation_config
            )
            
            if not response.candidates:
                raise Exception("No candidates in response")
            
            candidate = response.candidates[0]
            if not candidate.content or not candidate.content.parts:
                raise Exception(f"No valid response. Finish reason: {candidate.finish_reason}")

            text_content = response.text.replace('\xa0', ' ').strip()
            if not text_content:
                raise Exception("Empty text response")
            
            print(f"   └─ ✅ {label} complete")
            return text_content
            
        except Exception as e:
            print(f"   └─ ❌ {label} failed: {str(e)}")
            return None

    def process_pdf(self, pdf_path):
        """Process entire PDF file."""
        total_pages = self.get_pdf_page_count(pdf_path)
        print(f"   📄 Found {total_pages} page(s)")
        
        full_text = []
        successful_pages = 0
        
        for page_idx in range(total_pages):
            page_num = page_idx + 1
            page_bytes = self.extract_pdf_page(pdf_path, page_idx)
            text = self.process_content(page_bytes, 'application/pdf', f"page {page_num}")
            
            if text:
                if page_num == 1:
                    full_text.append(text)
                else:
                    full_text.append(f"\n\n--- Page {page_num} ---\n\n{text}")
                successful_pages += 1
            else:
                error_msg = f"[ERROR: Failed to process page {page_num}]"
                if page_num == 1:
                    full_text.append(error_msg)
                else:
                    full_text.append(f"\n\n--- Page {page_num} ---\n\n{error_msg}")
        
        return "".join(full_text), successful_pages, total_pages

    def process_image(self, image_path):
        """Process single image file."""
        ext = Path(image_path).suffix.lower()
        mime_type = MIME_TYPES.get(ext, 'image/jpeg')
        
        with open(image_path, 'rb') as f:
            image_bytes = f.read()
        
        text = self.process_content(image_bytes, mime_type, "image")
        return text, 1 if text else 0, 1


# ============================================
# PROCESSING BUTTON AND OUTPUT
# ============================================

ocr_output_area = widgets.Output()
ocr_results = {}

def run_ocr_process(b):
    global ocr_results
    ocr_results = {}
    
    with ocr_output_area:
        clear_output()
        
        # Get API key from widget or secrets
        api_key = api_key_input.value
        
        # If widget is empty, try to load from secrets dynamically
        if not api_key:
            try:
                from google.colab import userdata
                api_key = userdata.get('GEMINI_API_KEY')
                if api_key:
                    print("🔑 Found API key in Colab Secrets")
            except Exception:
                pass
        
        # Validate inputs
        if not api_key or len(api_key) < 20:
            print("❌ Please enter a valid API key in Step 2")
            print("   (Or add 'GEMINI_API_KEY' to Colab Secrets and re-run Step 2)")
            return
        
        if not uploaded_files:
            print("❌ Please upload at least one file in Step 3")
            return
        
        # Get settings
        model = model_dropdown.value
        
        # Get prompt
        custom_prompt = None
        prompt_file = None
        if use_custom_prompt.value and custom_prompt_text.value.strip():
            custom_prompt = custom_prompt_text.value.strip()
            print("📝 Using custom prompt")
        else:
            prompt_file = prompt_dropdown.value
            print(f"📝 Using prompt: {prompt_file}")
        
        print(f"🤖 Model: {model}")
        if "3-pro" in model:
            print("🧠 Thinking: level=LOW | Temperature: 0.1")
        else:
            print("⚡ Thinking: level=MINIMAL | Temperature: 0.2")
        print("🖼️ Media resolution: HIGH")
        print("\n" + "="*50)
        
        try:
            # Initialize OCR
            ocr = GeminiOCR(api_key, model, prompt_file, custom_prompt)
            print("✅ Connected to Gemini API\n")
            
            # Process each file
            for i, file_path in enumerate(uploaded_files, 1):
                filename = Path(file_path).name
                ext = Path(file_path).suffix.lower()
                
                is_pdf = ext == '.pdf'
                file_type = "PDF" if is_pdf else "Image"
                
                print(f"\n{'📄' if is_pdf else '🖼️'} Processing {file_type} {i}/{len(uploaded_files)}: {filename}")
                print("-" * 40)
                
                try:
                    if is_pdf:
                        final_text, successful, total = ocr.process_pdf(file_path)
                    else:
                        final_text, successful, total = ocr.process_image(file_path)
                    
                    # Save result
                    output_filename = Path(file_path).stem + "_ocr.txt"
                    output_path = os.path.join(FOLDERS['results'], output_filename)
                    
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(f"OCR/HTR Result\n")
                        f.write(f"Source: {filename}\n")
                        f.write(f"Model: {model}\n")
                        f.write(f"Prompt: {prompt_file if prompt_file else 'Custom'}\n")
                        f.write("=" * 50 + "\n\n")
                        f.write(final_text if final_text else "[No text extracted]")
                    
                    ocr_results[output_filename] = {'path': output_path}

                    # Auto-save to Google Drive if enabled
                    if DRIVE_MOUNTED and drive_save_enabled.value:
                        try:
                            drive_output_path = get_drive_output_folder()
                            if drive_output_path:
                                drive_file_path = os.path.join(drive_output_path, output_filename)
                                shutil.copy2(output_path, drive_file_path)
                                print(f"   ☁️ Saved to Drive: {DRIVE_OUTPUT_FOLDER}/{output_filename}")
                        except Exception as e:
                            print(f"   ⚠️ Could not save to Drive: {e}")
                    
                    print(f"\n✅ Complete! ({successful}/{total} {'pages' if is_pdf else 'processed'})")
                    print(f"   📄 Saved to: {output_path}")
                    
                except Exception as e:
                    print(f"\n❌ Error processing {filename}: {str(e)}")
            
            # Summary
            print("\n" + "="*50)
            print("🎉 PROCESSING COMPLETE!")
            print(f"   Files processed: {len(ocr_results)}")
            print(f"   📁 Local folder: {FOLDERS['results']}/")
            if DRIVE_MOUNTED and drive_save_enabled.value:
                print(f"   ☁️ Drive folder: My Drive/{DRIVE_OUTPUT_FOLDER}/")
            print("\n👇 Download your results in the next step")
            
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")

ocr_button = widgets.Button(
    description='🚀 Start OCR/HTR Processing',
    button_style='success',
    layout=widgets.Layout(width='250px', height='50px')
)
ocr_button.on_click(run_ocr_process)

display(ocr_button)
display(HTML("<br>"))
display(ocr_output_area)

## Step 6: Download Results 📥

Download the extracted text files to your computer.

In [ ]:
# ============================================
# DOWNLOAD RESULTS
# ============================================

download_output = widgets.Output()

def download_results(b):
    with download_output:
        clear_output()
        
        if not ocr_results:
            print("❌ No results available yet. Please run Step 5 first.")
            return
        
        print("📥 Preparing downloads...\n")
        
        for filename, data in ocr_results.items():
            try:
                filepath = data['path']
                print(f"   Downloading: {filename}")
                files.download(filepath)
            except Exception as e:
                print(f"   ⚠️ Could not download {filename}: {e}")
        
        print("\n✅ Downloads initiated! Check your browser's download folder.")

def download_all_zip(b):
    """Zip and download all results."""
    with download_output:
        clear_output()
        
        results_path = Path(FOLDERS['results'])
        txt_files = list(results_path.glob('*.txt'))
        
        if not txt_files:
            print("❌ No result files found.")
            return
        
        print(f"📦 Zipping {len(txt_files)} file(s)...")
        shutil.make_archive('ocr_results', 'zip', results_path)
        
        print("📥 Downloading zip file...")
        files.download('ocr_results.zip')
        print("\n✅ Download initiated!")

download_button = widgets.Button(
    description='📥 Download Latest Results',
    button_style='info',
    layout=widgets.Layout(width='250px', height='40px')
)
download_button.on_click(download_results)

download_zip_button = widgets.Button(
    description='📦 Download All as ZIP',
    button_style='',
    layout=widgets.Layout(width='250px', height='40px')
)
download_zip_button.on_click(download_all_zip)

display(widgets.HBox([download_button, download_zip_button]))
display(HTML(f"<br><i>💡 All results are saved in <code>{FOLDERS['results']}/</code></i>"))
display(download_output)

## Step 7: Cleanup 🧹

Delete uploaded files (PDFs/images) or results when you're done.

In [ ]:
cleanup_output = widgets.Output()

def cleanup_uploads(b):
    """Delete uploaded PDFs and images."""
    with cleanup_output:
        clear_output()
        count = 0
        # Use the single input folder defined in FOLDERS['input']
        path = Path(FOLDERS['input'])
        if path.exists():
            files_deleted = list(path.glob('*'))
            for f in files_deleted:
                if f.is_file():
                    f.unlink()
                    count += 1
        
        if count > 0:
            print(f"🧹 Deleted {count} uploaded file(s) from {FOLDERS['input']}/")
            global uploaded_files
            uploaded_files = []
        else:
            print(f"📁 Upload folder ({FOLDERS['input']}/) is already empty")

def cleanup_results(b):
    """Delete OCR/HTR results."""
    with cleanup_output:
        clear_output()
        path = Path(FOLDERS['results'])
        if path.exists():
            files_deleted = list(path.glob('*'))
            for f in files_deleted:
                if f.is_file():
                    f.unlink()
            if files_deleted:
                print(f"🧹 Deleted {len(files_deleted)} result file(s)")
                global ocr_results
                ocr_results = {}
            else:
                print("📁 Results folder is already empty")
        else:
            print("📁 Results folder is already empty")

def cleanup_all(b):
    """Delete all temporary files."""
    with cleanup_output:
        clear_output()
        cleanup_uploads(None)
        cleanup_results(None)
        print("\n✨ All temporary files cleared!")

btn_uploads = widgets.Button(
    description='🗑️ Delete Uploads', 
    button_style='warning', 
    layout=widgets.Layout(width='180px')
)
btn_res = widgets.Button(
    description='🗑️ Delete Results', 
    button_style='warning', 
    layout=widgets.Layout(width='180px')
)
btn_all = widgets.Button(
    description='🗑️ Delete Everything', 
    button_style='danger', 
    layout=widgets.Layout(width='180px')
)

btn_uploads.on_click(cleanup_uploads)
btn_res.on_click(cleanup_results)
btn_all.on_click(cleanup_all)

display(HTML("<b>🧹 Cleanup options:</b>"))
display(widgets.HBox([btn_uploads, btn_res, btn_all]))
display(cleanup_output)

---

### ☁️ Google Drive Integration

**Why use Google Drive?**
- OCR results are saved automatically to Drive
- If the browser disconnects, your work is preserved!
- Load PDFs/images directly from Drive

**Setup:**
1. Run Step 2.5 and click "☁️ Connect Google Drive"
2. Authorize access when prompted by Google
3. Optionally change the save folder name

**Auto-save location:** `My Drive/Colab_OCR_Results/`

---

### About

**OCR/HTR Colab Notebook** – Extract text from PDFs and images using Google's Gemini AI.

- **OCR:** Optical Character Recognition for printed documents
- **HTR:** Handwritten Text Recognition for manuscripts (French, Arabic, Multilingual)

**ZMO AI Pipelines** created by [Frédérick Madore](https://www.frederickmadore.com/)

Part of the [Leibniz-Zentrum Moderner Orient (ZMO)](https://www.zmo.de/) research tools.